In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

### подгтовка данных

In [4]:
ds = pd.read_csv('../data/r_1_SPB_clear_w_time_to_m.csv',index_col=0)

In [5]:
ds.columns

Index(['id', 'floor', 'floors_count', 'total_meters', 'price_per_month',
       'district', 'underground', 'Холодильник', 'Стиральная машина',
       'Телевизор', 'Посудомоечная машина', 'Кондиционер', 'Интернет',
       'Площадь кухни', 'Высота потолков', 'Санузел', 'Балкон/лоджия',
       'Вид из окон', 'Ремонт', 'Год постройки', 'Тип дома', 'Парковка',
       'adress', 'time_to_rent', 'm_lat', 'm_long', 'a_lat', 'a_long',
       'time_to_metro'],
      dtype='object')

удаляем координаты дома и ближайшего метро, округляем расстояние до метро до целых минут и удаляем стиральную машинку из-за высокой корреляции с холодильником, а так же район из-за высокой кореляции с метро в этом районе.

In [6]:
ds_reg = ds.drop(['a_lat','a_long','m_lat','m_long','adress','Стиральная машина','district','id'],axis = 1)

In [7]:
#todo смотреть на балконы в анализе и удалять там

In [8]:
ds_reg = ds_reg.drop(ds_reg[ds_reg['Балкон/лоджия'].isin(['2 лоджии','2 балкона'])].index)

In [9]:
ds_reg.reset_index(inplace=True, drop=True)

In [10]:
ds_reg.columns

Index(['floor', 'floors_count', 'total_meters', 'price_per_month',
       'underground', 'Холодильник', 'Телевизор', 'Посудомоечная машина',
       'Кондиционер', 'Интернет', 'Площадь кухни', 'Высота потолков',
       'Санузел', 'Балкон/лоджия', 'Вид из окон', 'Ремонт', 'Год постройки',
       'Тип дома', 'Парковка', 'time_to_rent', 'time_to_metro'],
      dtype='object')

In [11]:
ds_reg['time_to_metro'] = ds_reg['time_to_metro'].apply(lambda x: round(x))

In [12]:
ds_reg['first_floor'] = ds_reg['floor'].apply(lambda x: 1 if x == 1 else 0)

In [13]:
ds_reg.price_per_month = np.log(ds_reg.price_per_month.values)

In [14]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds_reg)*0.85)]
ds_test = ds_shuffled[int(len(ds_reg)*0.85):]

In [15]:
Y_train = ds_train['price_per_month']
X_train = ds_train.drop(['price_per_month','time_to_rent'],axis = 1)

In [16]:
Y_test = ds_test['price_per_month']
X_test = ds_test.drop(['price_per_month','time_to_rent'],axis = 1)

In [17]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
bin_cols

['Холодильник',
 'Телевизор',
 'Посудомоечная машина',
 'Кондиционер',
 'Интернет',
 'Санузел',
 'first_floor']

In [18]:
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
num_cols

['floor',
 'floors_count',
 'total_meters',
 'Площадь кухни',
 'Высота потолков',
 'Год постройки',
 'time_to_metro']

In [19]:
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)
cat_cols

['underground',
 'Балкон/лоджия',
 'Вид из окон',
 'Ремонт',
 'Тип дома',
 'Парковка']

In [20]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [21]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((690, 78), (122, 78))

In [22]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])

In [23]:
X_train = pd.DataFrame(X_train_norm.toarray(),columns=col_names)
X_test = pd.DataFrame(X_test_norm.toarray(),columns=col_names)
display(X_train,X_test)

,underground_Автово,underground_Академическая,underground_Балтийская,underground_Беговая,underground_Василеостровская,underground_Выборгская,underground_Гражданский проспект,underground_Девяткино,underground_Дунайская,underground_Елизаровская,...,Высота потолков,Год постройки,time_to_metro,Холодильник,Телевизор,Посудомоечная машина,Кондиционер,Интернет,Санузел,first_floor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.500000,1974.0,11.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.000000,2016.0,3.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.600000,1977.0,16.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,2022.0,54.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.000000,2004.0,15.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.500000,2016.0,40.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
686,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.672774,2003.0,13.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.700000,2014.0,20.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
688,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.672774,2020.0,72.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


,underground_Автово,underground_Академическая,underground_Балтийская,underground_Беговая,underground_Василеостровская,underground_Выборгская,underground_Гражданский проспект,underground_Девяткино,underground_Дунайская,underground_Елизаровская,...,Высота потолков,Год постройки,time_to_metro,Холодильник,Телевизор,Посудомоечная машина,Кондиционер,Интернет,Санузел,first_floor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,2006.0,13.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,1914.0,21.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.700000,2015.0,90.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.600000,1974.0,22.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,1987.0,10.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,2011.0,14.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.000000,2022.0,42.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
119,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.672774,1990.0,12.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.800000,2008.0,11.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0


---

### регрессия

In [24]:
import statsmodels.api as sm

In [25]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [26]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 78, but rank is 73
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_per_month   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                 4.907e+04
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        02:49:02   Log-Likelihood:                 337.25
No. Observations:                 690   AIC:                            -528.5
Df Residuals:                     617   BIC:                            -197.3
Df Model:                          72                                         
Covariance Type:                  HC3                                         
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                       1.5967      0.551      2.899      0.004       0.517       2.676
underground_Автово                         -0.1088      0.070     -1.548      0.122      -0.247       0.029
underground_Академическая                  -0.1250      0.050     -2.503      0.012      -0.223      -0.027
underground_Балтийская                      0.3398      0.132      2.575      0.010       0.081       0.598
underground_Беговая                         0.0702      0.071      0.993      0.321      -0.068       0.209
underground_Василеостровская                0.2793      0.077      3.647      0.000       0.129       0.429
underground_Выборгская                      0.0657      0.080      0.824      0.410      -0.090       0.222
underground_Гражданский проспект           -0.0771      0.050     -1.551      0.121      -0.175       0.020
underground_Девяткино                      -0.1183      0.074     -1.598      0.110      -0.264       0.027
underground_Дунайская                      -0.0212      0.058     -0.366      0.714      -0.135       0.092
underground_Елизаровская                   -0.0456      0.069     -0.659      0.510      -0.181       0.090
underground_Звездная                       -0.0934      0.052     -1.799      0.072      -0.195       0.008
underground_Комендантский проспект         -0.0838      0.047     -1.778      0.075      -0.176       0.009
underground_Купчино                        -0.0938      0.072     -1.305      0.192      -0.235       0.047
underground_Ладожская                       0.0467      0.065      0.719      0.472      -0.081       0.174
underground_Ленинский проспект             -0.0523      0.052     -0.999      0.318      -0.155       0.050
underground_Лесная                          0.1236      0.054      2.284      0.022       0.018       0.230
underground_Ломоносовская                  -0.0422      0.069     -0.613      0.540      -0.177       0.093
underground_Международная                   0.0798      0.054      1.478      0.140      -0.026       0.186
underground_Московская                      0.0280      0.049      0.567      0.571      -0.069       0.125
underground_Московские ворота               0.2843      0.083      3.414      0.001       0.121       0.448
underground_Нарвская                        0.1391      0.119      1.165      0.244      -0.095       0.373
underground_Новочеркасская                  0.1537      0.075      2.062      0.039       0.008       0.300
underground_Обводный канал                  0.1560      0.066      2.346      0.019       0.026       0.286
underground_Озерки                         -0.0852      0.065     -1.319      0.187      -0.212       0.041
underground_Парк Победы                     0.0784      0.0

In [27]:
X_test = sm.add_constant(X_test,has_constant='add')
preds_ts = results.predict(X_test)

In [28]:
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(Y_test,preds_ts)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(Y_test,preds_ts)))
print('Тестовый r2: {}'.format(r2_score(Y_test,preds_ts)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(Y_test,preds_ts)/Y_test.mean())*100,2)))
print('Тестовая средняя цена: {}'.format(Y_test.mean()))
print('Предсказанная средняя цена: {}'.format(preds_ts.mean()))

Тестовая средння абсолютная ошибка: 0.13280094857617583
Тестовая среднеквадратичная ошибка: 0.02852533036220662
Тестовый r2: 0.6767140835172855
Тестовое отношение mae к среднему: 1.28%
Тестовая средняя цена: 10.389780335624057
Предсказанная средняя цена: 10.403093847289952


---

Этаж получился спорным, так как либо при съеме это не влияет, либо, что равновероятно, там нелинейная зависимость, так как самые ценные этажи обычно находятся в середине дома

удалим стобец с метро, так как он выглядит незначимым

---

<!-- Удалим столбец с районами, так как они стат не значимые и несут в себе примерно такую же инфу, что и метро только более обощенную -->

In [29]:
ds_reg = ds_reg.drop(['underground'],axis = 1)

In [30]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds_reg)*0.85)]
ds_test = ds_shuffled[int(len(ds_reg)*0.85):]

In [31]:
Y_train = ds_train['price_per_month']
X_train = ds_train.drop(['price_per_month','time_to_rent'],axis = 1)
Y_test = ds_test['price_per_month']
X_test = ds_test.drop(['price_per_month','time_to_rent'],axis = 1)

In [32]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)

In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [34]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((690, 33), (122, 33))

In [35]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])
X_train = pd.DataFrame(X_train_norm,columns=col_names)
X_test = pd.DataFrame(X_test_norm,columns=col_names)

In [36]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [37]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 33, but rank is 29
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_per_month   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                 8.195e+04
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        02:49:03   Log-Likelihood:                 202.52
No. Observations:                 690   AIC:                            -347.0
Df Residuals:                     661   BIC:                            -215.5
Df Model:                          28                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                1.6498      0.730      2.260      0.024       0.219       3.080
Балкон/лоджия_1 балкон               0.4248      0.185      2.301      0.021       0.063       0.787
Балкон/лоджия_1 лоджия               0.4302      0.186      2.307      0.021       0.065       0.796
Балкон/лоджия_1 лоджия, 1 балкон     0.3667      0.192      1.912      0.056      -0.009       0.743
Балкон/лоджия_нет балкона            0.4282      0.182      2.349      0.019       0.071       0.786
Вид из окон_Во двор                  0.5158      0.244      2.118      0.034       0.038       0.993
Вид из окон_На улицу                 0.5683      0.244      2.325      0.020       0.089       1.047
Вид из окон_На улицу и двор          0.5657      0.244      2.315      0.021       0.087       1.045
Ремонт_Дизайнерский                  0.6754      0.247      2.736      0.006       0.192       1.159
Ремонт_Евроремонт                    0.5689      0.245      2.327      0.020       0.090       1.048
Ремонт_Косметический                 0.4054      0.240      1.691      0.091      -0.065       0.875
Тип дома_Блочный                     0.3488      0.150      2.319      0.020       0.054       0.644
Тип дома_Кирпичный                   0.3138      0.144      2.185      0.029       0.032       0.595
Тип дома_Монолитно-кирпичный         0.3851      0.149      2.583      0.010       0.093       0.677
Тип дома_Монолитный                  0.3429      0.152      2.260      0.024       0.045       0.640
Тип дома_Панельный                   0.2591      0.144      1.794      0.073      -0.024       0.542
Парковка_Многоуровневая              0.4012      0.187      2.143      0.032       0.034       0.768
Парковка_Наземная                    0.3772      0.179      2.110      0.035       0.027       0.728
Парковка_Открытая                    0.4126      0.186      2.224      0.026       0.049       0.776
Парковка_Подземная                   0.4587      0.185      2.483      0.013       0.097       0.821
floor                               -0.0002      0.002     -0.094      0.925      -0.003       0.003
floors_count                        -0.0110      0.002     -5.962      0.000      -0.015      -0.007
total_meters                         0.0042      0.002      1.858      0.063      -0.000       0.009
Площадь кухни                        0.0123      0.003      3.715      0.000       0.006       0.019
Высота потолков                      0.2889      0.090      3.210      0.001       0.113       0.465
Год постройки                        0.0027      0.001      3.002      0.003       0.001       0.005
time_to_metro                       -0.0035      0.000     -9.887      0.000      -0.004      -0.003
Холодильник                          0.1904      0.05

F-статистика,Log-Likelihood говорят, что мы все сделали правильно, хотя информационные критерии говорят иначе, но так как потом мы все равно будем сравнить с ГВР, где не будет метро, то на все равно нужна эта модель

<!-- #### Как и предполагалось r^2 слегка упал, зато большинство станций метро стали стат. значимыми -->

удалим этаж и санузел

In [38]:
ds_reg = ds_reg.drop(['Санузел','floor'],axis = 1)

In [39]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds_reg)*0.85)]
ds_test = ds_shuffled[int(len(ds_reg)*0.85):]

In [40]:
Y_train = ds_train['price_per_month']
X_train = ds_train.drop(['price_per_month','time_to_rent'],axis = 1)
Y_test = ds_test['price_per_month']
X_test = ds_test.drop(['price_per_month','time_to_rent'],axis = 1)

In [41]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [43]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((690, 31), (122, 31))

In [44]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])
X_train = pd.DataFrame(X_train_norm,columns=col_names)
X_test = pd.DataFrame(X_test_norm,columns=col_names)

In [45]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [46]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 31, but rank is 27
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_per_month   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                 8.862e+04
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        02:49:03   Log-Likelihood:                 202.45
No. Observations:                 690   AIC:                            -350.9
Df Residuals:                     663   BIC:                            -228.4
Df Model:                          26                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                1.6599      0.728      2.279      0.023       0.233       3.087
Балкон/лоджия_1 балкон               0.4274      0.184      2.322      0.020       0.067       0.788
Балкон/лоджия_1 лоджия               0.4323      0.186      2.325      0.020       0.068       0.797
Балкон/лоджия_1 лоджия, 1 балкон     0.3697      0.192      1.928      0.054      -0.006       0.745
Балкон/лоджия_нет балкона            0.4305      0.182      2.368      0.018       0.074       0.787
Вид из окон_Во двор                  0.5195      0.243      2.137      0.033       0.043       0.996
Вид из окон_На улицу                 0.5715      0.244      2.345      0.019       0.094       1.049
Вид из окон_На улицу и двор          0.5689      0.244      2.333      0.020       0.091       1.047
Ремонт_Дизайнерский                  0.6788      0.246      2.756      0.006       0.196       1.162
Ремонт_Евроремонт                    0.5723      0.244      2.347      0.019       0.094       1.050
Ремонт_Косметический                 0.4088      0.239      1.708      0.088      -0.060       0.878
Тип дома_Блочный                     0.3504      0.150      2.340      0.019       0.057       0.644
Тип дома_Кирпичный                   0.3159      0.143      2.204      0.028       0.035       0.597
Тип дома_Монолитно-кирпичный         0.3875      0.149      2.607      0.009       0.096       0.679
Тип дома_Монолитный                  0.3448      0.152      2.275      0.023       0.048       0.642
Тип дома_Панельный                   0.2612      0.144      1.811      0.070      -0.022       0.544
Парковка_Многоуровневая              0.4037      0.187      2.161      0.031       0.037       0.770
Парковка_Наземная                    0.3798      0.178      2.128      0.033       0.030       0.730
Парковка_Открытая                    0.4148      0.185      2.242      0.025       0.052       0.777
Парковка_Подземная                   0.4616      0.184      2.505      0.012       0.100       0.823
floors_count                        -0.0111      0.002     -6.948      0.000      -0.014      -0.008
total_meters                         0.0043      0.002      1.879      0.060      -0.000       0.009
Площадь кухни                        0.0123      0.003      3.756      0.000       0.006       0.019
Высота потолков                      0.2882      0.090      3.209      0.001       0.112       0.464
Год постройки                        0.0028      0.001      3.016      0.003       0.001       0.005
time_to_metro                       -0.0035      0.000     -9.883      0.000      -0.004      -0.003
Холодильник                          0.1909      0.057      3.370      0.001       0.080       0.302
Телевизор                            0.0317      0.01

Остальные коэффициенты стали еще чуть более значимыми, F-статистика и информациооные коэфы подросли

Тот факт, что наличие интернета отрицательно влияет на стоимость не несет в себе никакого экономеческого смысла, поэтому его тоже удалим

In [47]:
ds_reg = ds_reg.drop(['Интернет'],axis = 1)

In [48]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds_reg)*0.85)]
ds_test = ds_shuffled[int(len(ds_reg)*0.85):]

In [49]:
Y_train = ds_train['price_per_month']
X_train = ds_train.drop(['price_per_month','time_to_rent'],axis = 1)
Y_test = ds_test['price_per_month']
X_test = ds_test.drop(['price_per_month','time_to_rent'],axis = 1)

In [50]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)

In [51]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [52]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((690, 30), (122, 30))

In [53]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])
X_train = pd.DataFrame(X_train_norm,columns=col_names)
X_test = pd.DataFrame(X_test_norm,columns=col_names)

In [54]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [55]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 30, but rank is 26
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_per_month   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                 9.152e+04
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        02:49:03   Log-Likelihood:                 202.02
No. Observations:                 690   AIC:                            -352.0
Df Residuals:                     664   BIC:                            -234.1
Df Model:                          25                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                1.6549      0.732      2.260      0.024       0.220       3.090
Балкон/лоджия_1 балкон               0.4250      0.185      2.299      0.022       0.063       0.787
Балкон/лоджия_1 лоджия               0.4299      0.187      2.301      0.021       0.064       0.796
Балкон/лоджия_1 лоджия, 1 балкон     0.3720      0.192      1.937      0.053      -0.004       0.748
Балкон/лоджия_нет балкона            0.4279      0.183      2.345      0.019       0.070       0.786
Вид из окон_Во двор                  0.5180      0.244      2.120      0.034       0.039       0.997
Вид из окон_На улицу                 0.5702      0.245      2.327      0.020       0.090       1.051
Вид из окон_На улицу и двор          0.5667      0.245      2.312      0.021       0.086       1.047
Ремонт_Дизайнерский                  0.6762      0.248      2.731      0.006       0.191       1.161
Ремонт_Евроремонт                    0.5700      0.245      2.326      0.020       0.090       1.050
Ремонт_Косметический                 0.4087      0.241      1.698      0.090      -0.063       0.880
Тип дома_Блочный                     0.3498      0.151      2.323      0.020       0.055       0.645
Тип дома_Кирпичный                   0.3148      0.144      2.184      0.029       0.032       0.597
Тип дома_Монолитно-кирпичный         0.3862      0.149      2.585      0.010       0.093       0.679
Тип дома_Монолитный                  0.3443      0.152      2.259      0.024       0.046       0.643
Тип дома_Панельный                   0.2599      0.145      1.793      0.073      -0.024       0.544
Парковка_Многоуровневая              0.4027      0.188      2.143      0.032       0.034       0.771
Парковка_Наземная                    0.3796      0.179      2.115      0.034       0.028       0.731
Парковка_Открытая                    0.4122      0.186      2.218      0.027       0.048       0.776
Парковка_Подземная                   0.4603      0.185      2.485      0.013       0.097       0.823
floors_count                        -0.0111      0.002     -6.945      0.000      -0.014      -0.008
total_meters                         0.0041      0.002      1.822      0.068      -0.000       0.009
Площадь кухни                        0.0122      0.003      3.750      0.000       0.006       0.019
Высота потолков                      0.2851      0.090      3.185      0.001       0.110       0.461
Год постройки                        0.0028      0.001      3.008      0.003       0.001       0.005
time_to_metro                       -0.0035      0.000     -9.814      0.000      -0.004      -0.003
Холодильник                          0.1874      0.056      3.375      0.001       0.079       0.296
Телевизор                            0.0311      0.01

---

In [56]:
X_test = sm.add_constant(X_test,has_constant='add')
preds_ts = results.predict(X_test)

In [57]:
print('Тестовая средння абсолютная ошибка: {}'.format(mean_absolute_error(Y_test,preds_ts)))
print('Тестовая среднеквадратичная ошибка: {}'.format(mean_squared_error(Y_test,preds_ts)))
print('Тестовый r2: {}'.format(r2_score(Y_test,preds_ts)))
print('Тестовое отношение mae к среднему: {}%'.format(round((mean_absolute_error(Y_test,preds_ts)/Y_test.mean())*100,2)))
print('Тестовая средняя цена: {}'.format(Y_test.mean()))
print('Предсказанная средняя цена: {}'.format(preds_ts.mean()))

Тестовая средння абсолютная ошибка: 0.14724257609847943
Тестовая среднеквадратичная ошибка: 0.03690821398478084
Тестовый r2: 0.5817084103040339
Тестовое отношение mae к среднему: 1.42%
Тестовая средняя цена: 10.389780335624057
Предсказанная средняя цена: 10.402436572260665


Модель обладает обощающей способностью

к удалению: 'Интернет', 'Санузел','floor','underground','Стиральная машина','district'

---

посмотрим, может быть районы значимы

In [ ]:
ds = pd.read_csv('../data/r_1_SPB_clear_w_time_to_m.csv',index_col=0)

In [ ]:
ds_reg = ds.drop(['a_lat','a_long','m_lat','m_long','adress','Стиральная машина','underground','id'],axis = 1)
ds_reg = ds_reg.drop(ds_reg[ds_reg['Балкон/лоджия'].isin(['2 лоджии','2 балкона'])].index)
ds_reg.reset_index(inplace=True, drop=True)
ds_reg['time_to_metro'] = ds_reg['time_to_metro'].apply(lambda x: round(x))
ds_reg['first_floor'] = ds_reg['floor'].apply(lambda x: 1 if x == 1 else 0)
ds_reg.price_per_month = np.log(ds_reg.price_per_month.values)
ds_reg = ds_reg.drop(['Санузел','floor'],axis = 1)


In [ ]:
ds_shuffled = ds_reg.sample(len(ds_reg),random_state= 66).reset_index(drop=True)
ds_train = ds_shuffled[:int(len(ds_reg)*0.85)]
ds_test = ds_shuffled[int(len(ds_reg)*0.85):]

In [ ]:
Y_train = ds_train['price_per_month']
X_train = ds_train.drop(['price_per_month','time_to_rent'],axis = 1)
Y_test = ds_test['price_per_month']
X_test = ds_test.drop(['price_per_month','time_to_rent'],axis = 1)

In [ ]:
bin_cols = list(X_train.nunique()[X_train.nunique() == 2].index)
num_cols = list(X_train.drop(bin_cols,axis =1).select_dtypes(include = 'number').columns)
non_cat_cols = num_cols.copy()
non_cat_cols.extend(bin_cols)
cat_cols = list(X_train.drop(non_cat_cols,axis =1).columns)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat',OneHotEncoder() ,cat_cols),
        ('num', 'passthrough', num_cols),
        ('bin','passthrough',bin_cols)
    ])

In [ ]:
X_train_norm = preprocessor.fit_transform(X_train)
X_test_norm = preprocessor.transform(X_test)
X_train_norm.shape,X_test_norm.shape

((690, 45), (122, 45))

In [ ]:
col_names = []
for item in preprocessor.get_feature_names_out():
    col_names.append(item.split('__')[1])
X_train = pd.DataFrame(X_train_norm,columns=col_names)
X_test = pd.DataFrame(X_test_norm,columns=col_names)

In [ ]:
X_train = sm.add_constant(X_train,has_constant='add')
results= sm.OLS(Y_train,X_train).fit(cov_type = 'HC3')

In [ ]:
results.summary()

c:\Users\ntsec\anaconda3\envs\new_env\lib\site-packages\statsmodels\base\model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 45, but rank is 40
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_per_month   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                 7.065e+04
Date:                Mon, 04 Mar 2024   Prob (F-statistic):               0.00
Time:                        02:39:41   Log-Likelihood:                 259.44
No. Observations:                 690   AIC:                            -438.9
Df Residuals:                     650   BIC:                            -257.4
Df Model:                          39                                         
Covariance Type:                  HC3                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                1.6165      0.722      2.240      0.025       0.202       3.031
district_Адмиралтейский              0.3211      1.117      0.287      0.774      -1.868       2.510
district_Василеостровский            0.2825      1.116      0.253      0.800      -1.906       2.471
district_Выборгский                  0.0943      1.116      0.084      0.933      -2.093       2.282
district_Калининский                 0.0932      1.116      0.083      0.933      -2.094       2.280
district_Кировский                   0.0344      1.116      0.031      0.975      -2.154       2.223
district_Колпинский                 -0.2656     14.968     -0.018      0.986     -29.602      29.071
district_Красногвардейский           0.1607      1.116      0.144      0.886      -2.027       2.348
district_Красносельский              0.1128      1.116      0.101      0.920      -2.075       2.300
district_Московский                  0.1342      1.116      0.120      0.904      -2.053       2.322
district_Невский                     0.0446      1.116      0.040      0.968      -2.143       2.232
district_Приморский                  0.1129      1.116      0.101      0.919      -2.074       2.300
district_Пушкинский                 -0.0001      1.117     -0.000      1.000      -2.189       2.189
district_Фрунзенский                 0.1642      1.116      0.147      0.883      -2.023       2.352
district_Центральный                 0.3275      1.117      0.293      0.769      -1.862       2.516
Балкон/лоджия_1 балкон               0.4085      0.183      2.232      0.026       0.050       0.767
Балкон/лоджия_1 лоджия               0.4024      0.184      2.188      0.029       0.042       0.763
Балкон/лоджия_1 лоджия, 1 балкон     0.3997      0.194      2.065      0.039       0.020       0.779
Балкон/лоджия_нет балкона            0.4059      0.180      2.255      0.024       0.053       0.759
Вид из окон_Во двор                  0.5020      0.240      2.088      0.037       0.031       0.973
Вид из окон_На улицу                 0.5468      0.242      2.261      0.024       0.073       1.021
Вид из окон_На улицу и двор          0.5676      0.242      2.349      0.019       0.094       1.041
Ремонт_Дизайнерский                  0.6535      0.243      2.693      0.007       0.178       1.129
Ремонт_Евроремонт                    0.5574      0.241      2.309      0.021       0.084       1.031
Ремонт_Косметический                 0.4056      0.239      1.699      0.089      -0.062       0.873
Тип дома_Блочный                     0.3299      0.147      2.240      0.025       0.041       0.619
Тип дома_Кирпичный                   0.2895      0.143      2.026      0.043       0.009       0.570
Тип дома_Монолитно-кирпичный         0.3948      0.14

районы тоже незначимые